In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import onnxruntime as rt
import tensorflow as tf

In [ ]:
model_path = Path(
    "/path/to/model/suspicious_davinci_0a6593ee/onnx/suspicious_davinci_0a6593ee.onnx"
)

In [ ]:
session = rt.InferenceSession(str(model_path), providers=["CPUExecutionProvider"])

In [ ]:
for ix, i in enumerate(session.get_inputs()):
    print(f"input {ix}: {i}")

for ix, i in enumerate(session.get_outputs()):
    print(f"output {ix}: {i}")

# Keras Model

In [ ]:
model = tf.keras.models.load_model("/path/to/model/cranky_mahavira_ba989e30/keras/")
model.summary()

In [ ]:
img = cv2.imread(
    "../image.png",
    cv2.IMREAD_COLOR,
)
img = cv2.resize(img, (224, 224))

In [ ]:
pred = model.predict(img[np.newaxis, :, :, :])
pred_class = np.argmax(pred[0])
weights = model.layers[-1].get_weights()[0]
class_weights = weights[:, pred_class]

In [ ]:
pred_class_tf = tf.math.argmax(pred[0])

In [ ]:
pred_class_tf.shape

In [ ]:
first_layer = model.layers[2].get_input_at(0)
last_conv_layer = model.layers[-2].layers[-2].get_output_at(0)

In [ ]:
backbone = model.layers[-2]

In [ ]:
model.input

In [ ]:
model.inputs

In [ ]:
tf.keras.utils.plot_model(
    model, to_file="model.png", show_shapes=True, show_dtype=True, expand_nested=True
)

In [ ]:
model.layers

In [ ]:
backbone_conv_model = tf.keras.Model(
    inputs=model.layers[1].input, outputs=[last_conv_layer]
)

In [ ]:
new_inputs = tf.keras.layers.Input(shape=model.input.shape[1:])

x = backbone_conv_model(new_inputs)
new_model = tf.keras.Model(inputs=new_inputs, outputs=[x], name="castrato")

In [ ]:
out = new_model(img[np.newaxis, ...])

In [ ]:
out

In [ ]:
out[0].shape

In [ ]:
class_weights.reshape((1, 1, -1)).shape

In [ ]:
heatmaps = tf.image.resize(out[0], [224, 224], method="bilinear")

output = tf.tensordot(heatmaps, class_weights, axes=((2,), (0,)))

In [ ]:
output.shape

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(output)

In [ ]:
new_new_inp = tf.keras.layers.Input(shape=model.input.shape[1:])
pred = model(new_new_inp)
pred2 = new_model(new_new_inp)
pred_class = tf.math.argmax(pred, axis=1)
print(pred_class.shape)
print(pred_class)
weights = model.layers[-1].get_weights()[0]
print(weights.shape)
class_weights = tf.gather(weights, pred_class, axis=1)
print(class_weights.shape)

heatmaps = tf.image.resize(pred2, [224, 224], method="bilinear")

print(heatmaps.shape)

output = tf.tensordot(heatmaps, class_weights, axes=((3,), (0,)))

model_with_heatmap = tf.keras.Model(new_new_inp, [pred, output])

In [ ]:
pred_class

In [ ]:
def attach_heatmap(model):
    last_conv_layer = model.layers[-2].layers[-2].get_output_at(0)
    weights = model.layers[-1].get_weights()[0]

    backbone_conv_model = tf.keras.Model(
        inputs=model.layers[1].input, outputs=[last_conv_layer]
    )

    new_inputs = tf.keras.layers.Input(shape=model.input.shape[1:])
    conv_layer = backbone_conv_model(new_inputs)
    new_model = tf.keras.Model(inputs=new_inputs, outputs=[conv_layer], name="castrato")

    new_input = tf.keras.layers.Input(shape=model.input.shape[1:])
    pred = model(new_input)
    conv = new_model(new_input)
    pred_class = tf.math.argmax(pred, axis=1)
    class_weights = tf.gather(weights, pred_class, axis=1)

    heatmaps = tf.image.resize(conv, [224, 224], method="bilinear")

    output = tf.tensordot(heatmaps, class_weights, axes=((3,), (0,)))

    return tf.keras.Model(new_input, [pred, output])

In [ ]:
final_model = attach_heatmap(model)

In [ ]:
heat = model_with_heatmap(img[np.newaxis, ...])

In [ ]:
heat

In [ ]:
heat = final_model(img[np.newaxis, ...])

In [ ]:
plt.imshow(heat[1][0, ...])

In [ ]:
test_out = model_with_heatmap(np.zeros((10, 224, 224, 3)))

In [ ]:
test_out

In [ ]:
heat

In [ ]:
heat.shape

In [ ]:
plt.imshow(heat)